In [1]:
#### Load packages

import json
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [2]:
#### load data

with open('data/read_csv_kwargs.json', 'r') as fin:
    kwar = json.load(fin)
    
ts_nests = pd.read_csv('data/training_set_nest_counts.csv', **kwar)   
sub_format = pd.read_csv('data/submission_format.csv', **kwar)

names = ts_nests.reset_index().common_name.unique()

In [3]:
#### compute last valid value and copy it in the submission format

last_value = ts_nests.fillna(method='ffill', axis = 1)['2013'].fillna(value = 0.0)

for col in sub_format.columns:
    sub_format.ix[:,col] = last_value

In [4]:
#### specific change rate per penguin type

alpha = {names[0]: 0.900, 
         names[1]: 1.075, 
         names[2]: 1.075}      
print(alpha)

{'chinstrap penguin': 1.075, 'gentoo penguin': 1.075, 'adelie penguin': 0.9}


In [5]:
#### load change rates per penguin type, per year (although in this case are all the same)

beta = pd.read_csv('beta.csv', index_col=0)
print(beta)

In [6]:
#### apply rates and save submission

for i in range(len(sub_format)):
    penguin = sub_format.iloc[i].name[1]
    for year in sub_format.columns:
        fac = beta.ix[penguin, year]
        sub_format.iloc[i][year] = sub_format.iloc[i][year]*fac
        

csv = 'submissions/BCA_LV_{:.3f}_{:.3f}_{:.3f}.csv'.format(*[alpha[x] for x in names])
sub_format.to_csv(csv)